## Bulk update maps for Use in Collector / Field Maps

This notebook allows the user to define a search string for webmaps and set the ability to "Use in Field Maps" and/or "Use in Field Maps". It will first search for maps based on a string you define, confirm these are the maps you'd like to update, and then update each for Collector / FieldMaps use based on parameters you define. Additionally, this notebook can be imported in ArcGIS Online and used as a hosted notebook.

In [1]:
from arcgis.gis import GIS
gis = GIS("home")
# could also try: gis = GIS("https://arcgis.com","<username>", verify_cert=False) 

Enter password: ········


### Please set the parameters below. Example parameters have been set for this notebook.

In this example query, we'll disable in Field Maps all webmaps that are currently not enabled for Collector. This will prevent field workers from accidentally using the wrong map!

In [2]:
# this takes "enabled", "disabled", or "no_change"
field_maps_usage = "disabled"
collector_usage = "no_change"
webmap_search_query = "Hydrant NOT typekeywords: 'Collector' OR typekeywords: 'CollectorDisabled'"

# Example query which would only update maps currently enabled for Collector
# webmap_search_query = "title:Hydrant AND typekeyword:'Collector'"

# Example query which would update maps currently enabled for Field Maps
# webmap_search_query = "title:Hydrant AND NOT typekeyword:'FieldMapsDisabled'"

valid_entries = ["enabled", "disabled", "no_change"]
if not field_maps_usage in valid_entries or not collector_usage in valid_entries:
    raise Exception("Check your usage parameters are set correctly")

### Now, let's search for the webmaps that we're going to update

In [3]:
webmap_search_query = webmap_search_query + f" AND owner:{gis.users.me.username}"
webmaps = gis.content.search(webmap_search_query, item_type='Web Map', max_items=1000, outside_org=False)
print(webmaps)
print("\n")
if len(webmaps) > 0:
        yn = (input(str(len(webmaps)) + ' web maps found. Are you sure you want to work on all these web maps? (y/N): ') or 'n').lower().strip()
        sure = yn in ['yes', 'y', 'ye']
else:
    raise Exception("No items found with query:", webmap_search_query)

[<Item title:"Hydrant Inspections" type:Web Map owner:workforce_scripts>, <Item title:"Hydrant Inspections_dispatchers" type:Web Map owner:workforce_scripts>, <Item title:"Hydrant Inspection" type:Web Map owner:workforce_scripts>, <Item title:"Hydrant Inspections_workers" type:Web Map owner:workforce_scripts>]


4 web maps found. Are you sure you want to work on all these web maps? (y/N): y


### If you want to continue, we'll now update the keywords needed to use Collector and Field Maps

In [4]:
if not sure:
    raise Exception("Script terminated. Please check your search string again")
else:
    for wm in webmaps:
        print("Checking whether to update webmap for Field Maps and Collector: " + wm.title)
        current_keywords = set(wm.typeKeywords)
        
        if field_maps_usage == "enabled":
            current_keywords.discard("FieldMapsDisabled")
        elif field_maps_usage == "disabled":
            current_keywords.add("FieldMapsDisabled")

        if collector_usage == "enabled":
            current_keywords.discard("CollectorDisabled")
            current_keywords.add("Collector")
        elif collector_usage == "disabled":
            current_keywords.discard("Collector")
            current_keywords.add("CollectorDisabled")

        if sorted(list(current_keywords)) != sorted(wm.typeKeywords):
            print("New keyword added for " + wm.title + ". Updating webmap")
            try:
                wm.update(item_properties={"typeKeywords": list(current_keywords)})
            except Exception as e:
                print(e)
        else:
            print("Skipped updating webmap for " + wm.title)
        print("\n")
            
print("Done!")

Checking whether to update webmap for Field Maps: Hydrant Inspections
New keyword added for Hydrant Inspections. Updating webmap


Checking whether to update webmap for Field Maps: Hydrant Inspections_dispatchers
New keyword added for Hydrant Inspections_dispatchers. Updating webmap


Checking whether to update webmap for Field Maps: Hydrant Inspection
New keyword added for Hydrant Inspection. Updating webmap


Checking whether to update webmap for Field Maps: Hydrant Inspections_workers
New keyword added for Hydrant Inspections_workers. Updating webmap


Done!
